In [47]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<font color = "#CC3D3D"><p>
# Topics
* [Group Aggregation](#Group-Aggregation)
* [Pivot Tables](#Pivot-Tables)
* [Merging DataFrames](#Merging-DataFrames)
* [Appending DataFrames](#Appending-DataFrames)

## Group Aggregation
<br><img align="left" src="http://drive.google.com/uc?export=view&id=17lLj-fLLYk6Dxcz7yBIX7bMEl4PAESBB" width=800 height=600>

In [4]:
df = DataFrame({'key'  : ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                'data' : [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

,key,data
0,A,0
1,B,5
2,C,10
3,A,5
4,B,10
5,C,15
6,A,10
7,B,15
8,C,20


<font color = 'blue'>Typical usage

In [5]:
df.groupby('key')['data'].sum()

key
A    15
B    30
C    45
Name: data, dtype: int64

In [6]:
df.groupby('key')['data'].size()  # size()는 결측값 포함해서 빈도 측정, count()는 결측값 제외 

key
A    3
B    3
C    3
Name: data, dtype: int64

In [7]:
# When we reset the index, the old index is added as a column, and a new sequential index is used
df.groupby('key')['data'].sum().reset_index()

,key,data
0,A,15
1,B,30
2,C,45


In [8]:
df.groupby('key', as_index=False)['data'].sum()

,key,data
0,A,15
1,B,30
2,C,45


<font color = 'blue'>Iterating through groups

In [9]:
grouped = df.groupby('key')['data']



for name, group in grouped:
    print(name)
    print(group)

A
0     0
3     5
6    10
Name: data, dtype: int64
B
1     5
4    10
7    15
Name: data, dtype: int64
C
2    10
5    15
8    20
Name: data, dtype: int64


In [10]:
[v for v in grouped.get_group('A')]

[0, 5, 10]

<font color = 'blue'>Filtration

In [11]:
print(df)
df.groupby('key')['data'].sum()

  key  data
0   A     0
1   B     5
2   C    10
3   A     5
4   B    10
5   C    15
6   A    10
7   B    15
8   C    20


key
A    15
B    30
C    45
Name: data, dtype: int64

In [12]:
df.groupby('key')['data'].filter(lambda x: x.sum() > 30)

2    10
5    15
8    20
Name: data, dtype: int64

<font color = 'blue'>Using agg method

In [13]:
df.groupby('key')['data'].agg('max')
#df.groupvy('key')['data'].max()

key
A    10
B    15
C    20
Name: data, dtype: int64

In [14]:
df.groupby('key')['data'].agg(lambda x: x.max() - x.min())

key
A    10
B    10
C    10
Name: data, dtype: int64

In [15]:
def max_min(x):
    return x.max() -x.min()

df.groupby('key')['data'].agg(max_min)
#위에 꺼랑 같은데 이거는 함수 선언으로 표현 

key
A    10
B    10
C    10
Name: data, dtype: int64

<font color = 'blue'>Applying multiple functions at once

In [16]:
df.groupby('key')['data'].agg(['mean', 'std'])

,mean,std
key,,
A,5,5.0
B,10,5.0
C,15,5.0


<font color = 'blue'>Named aggregation

In [17]:
df.groupby('key')['data'].agg([('평균','mean'), ('표준편차','std')])

,평균,표준편차
key,,
A,5,5.0
B,10,5.0
C,15,5.0


<font color = 'blue'>By default, all of the numeric columns are aggregated.

In [18]:
df['average'] = df.data.apply(lambda x, y: 'above' if x > y else 'below', args=(df.data.mean(),))
df

,key,data,average
0,A,0,below
1,B,5,below
2,C,10,below
3,A,5,below
4,B,10,below
5,C,15,above
6,A,10,below
7,B,15,above
8,C,20,above


In [19]:
df.groupby(['key']).mean()

,data
key,
A,5
B,10
C,15


<font color = 'blue'>Applying different functions to DataFrame columns

In [20]:
df.groupby('key').agg({'data' : [('평균','mean'), ('합계', np.sum)], 'average' : [('범주',Series.nunique)]})

data     average
      평균  합계      범주
key                 
A      5  15       1
B     10  30       2
C     15  45       2

## Pivot Tables
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1HEBp4qq4GaksdQBb2fx2tRsq-gVR-j6d" width=800 height=600>

In [21]:
tr = pd.DataFrame({
        'id': [1,1,1,1,2,2,2],
        'site': ['a','b','c','a','a','b','b'],
        'pageview': np.arange(1,8),
        'dwelltime': np.arange(7.0, 0, -1),
    }, columns=['id','site','pageview','dwelltime'])

tr 

,id,site,pageview,dwelltime
0,1,a,1,7.0
1,1,b,2,6.0
2,1,c,3,5.0
3,1,a,4,4.0
4,2,a,5,3.0
5,2,b,6,2.0
6,2,b,7,1.0


In [22]:
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum)

site,a,b,c
id,,,
1,5.0,2.0,3.0
2,5.0,13.0,NaN


In [23]:
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum, fill_value=0) # 결측값은 0 으로 지정 

site,a,b,c
id,,,
1,5,2,3
2,5,13,0


In [24]:
pd.pivot_table(tr, values='pageview', index='id', columns='site', 
               aggfunc=sum, fill_value=0).reset_index()

site,id,a,b,c
0,1,5,2,3
1,2,5,13,0


In [25]:
pv = pd.pivot_table(tr, values='pageview', index='id', columns='site',
                    aggfunc=sum, fill_value=0).reset_index()
pv.columns.name = None; pv

,id,a,b,c
0,1,5,2,3
1,2,5,13,0


In [31]:
pd.pivot_table(tr, values='pageview', index='id', 
               aggfunc=np.mean, fill_value=0).reset_index()

,id,pageview
0,1,2.5
1,2,6.0


In [32]:
pd.pivot_table(tr, values='pageview', columns='site', 
               aggfunc=np.size, fill_value=0)

site,a,b,c
pageview,3,3,1


In [33]:
tr

,id,site,pageview,dwelltime
0,1,a,1,7.0
1,1,b,2,6.0
2,1,c,3,5.0
3,1,a,4,4.0
4,2,a,5,3.0
5,2,b,6,2.0
6,2,b,7,1.0


## Merging DataFrames

In [93]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [94]:
display(df1)
display(df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [95]:
pd.merge(df1, df2) #값이 공통이 애들만 추출됨 

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<font color = 'blue'>If not specified, merge uses the overlapping column names as the keys. It’s a good practice to specify explicitly, though:

In [96]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<font color = 'blue'>If the column names are different in each object, you can specify them separately:

In [97]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [100]:
print(df3)
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


In [98]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


<font color='darkgreen'><p>
##### Merge Types
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1yEU_xZ9qZrZi8IIWTMhhMIKI95XxisX0" width=900 height=800>

<font color = 'blue'>By default merge does an 'inner' join; the keys in the result are the intersection. The outer join takes the union of the keys:

In [114]:
pd.merge(df1, df2, how='outer') 

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


<font color = 'blue'>Pandas also provides the *merge* DataFrame method:

In [ ]:
df1.merge(df2, how='outer')

## Appending DataFrames
<br><img align="left" src="http://drive.google.com/uc?export=view&id=1lSpzDfbRlY_mTlJH0t1xZADJUvbHyPMw" width=800 height=600>

In [126]:
df5 = DataFrame({'key': ['a', 'b', 'd'], 'data1': range(3)})
print('df1:\n', df1)

print('df5:\n',df5)

df1:
   key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
df5:
   key  data1
0   a      0
1   b      1
2   d      2


In [127]:
pd.concat([df1, df5])

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
0,a,0
1,b,1
2,d,2


In [136]:
pd.concat([df1, df5]).reset_index(drop=True) 
#reset_index()로 재정렬 drop으로 인덱스 재정렬

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
7,a,0
8,b,1
9,d,2


In [138]:
pd.concat([df1, df5],ignore_index=True) 
#위의 것과 같은 거 도출 

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6
7,a,0
8,b,1
9,d,2


In [129]:
pd.concat([df1, df5], axis=1)  #axis=1은 좌우로 붙게해주는기능

,key,data1,key,data1
0,b,0,a,0.0
1,b,1,b,1.0
2,a,2,d,2.0
3,c,3,NaN,NaN
4,a,4,NaN,NaN
5,a,5,NaN,NaN
6,b,6,NaN,NaN


<font color = "blue"><p>
## Exercises

In [34]:
cs = pd.read_csv('Demo.csv', encoding='cp949', engine='python')
tr = pd.read_csv('구매내역정보.csv', encoding='cp949', engine='python')

**[연습문제 28]** `60대 여성` 고객리스트를 출력하시오.

In [161]:
cs.query('성별 ==2 and 59 < 연령 < 70 ')

,ID,성별,연령,거주지역
222,55844757,2,69,서울 서대문구
223,165639634,2,69,부산 동래구
225,969901888,2,69,서울 동대문구
226,779190802,2,69,서울 용산구
228,414448990,2,69,경기 안산시
...,...,...,...,...
3516,473266991,2,69,서울 강서구
3524,330939633,2,60,경기 안양시
3532,524493815,2,61,부산 수영구
3539,168692065,2,63,경기 안양시


**[연습문제 29]** `남성고객`과 `여성고객`은 각각 몇명인가?

In [54]:
cs.groupby('성별').size().reset_index()

,성별,0
0,1,859
1,2,2683


**[연습문제 30]** `여성고객`의 `평균나이`는 얼마인가?

In [30]:
#cs.groupby('성별')['연령'].mean().reset_index()


cs.groupby('성별')['연령'].agg([('평균나이',np.mean)]).reset_index() 
#두개 같은 것 


,성별,평균나이
0,1,62.442375
1,2,60.637719


In [92]:
#거주지역 알아보기 

cs.groupby('성별')['연령'].agg([('평균나이',np.mean),('거주지수',Series.nunique)]).reset_index() 

,성별,평균나이,거주지수
0,1,62.442375,30
1,2,60.637719,30


In [6]:
#고객별 총 구매액을 계산하세요 .
tr.groupby('ID')['구매금액'].sum().reset_index()

,ID,구매금액
0,741463,560000
1,909249,640000
2,1065068,1103000
3,1310410,249000
4,1387032,113000
...,...,...
3537,998623524,2088000
3538,999024423,25000
3539,999184958,3147000
3540,999375162,10762000


In [8]:
#고객별 총 구매액의 최고 금액을 출력 
tr.groupby('ID')['구매금액'].sum().reset_index().구매금액.max()

136348000

In [10]:
#고객별 가장 많이 구매한 건수 
tr.groupby('ID')['구매금액'].size().reset_index().구매금액.max()   

641

**[연습문제 31]** `70대 여성 고객`들은 주로 어느 `지역`에 거주하고 있는가? 상위 5개 지역만 나열하시오.

In [162]:
cs.query('성별 == 2 and 69< 연령 <80').거주지역.value_counts()[:5]

경기 고양시     20
서울 송파구     13
부산 해운대구    12
경기 성남시     12
경기 안양시      9
Name: 거주지역, dtype: int64

**[연습문제 32]** (상품중분류명 기준) `Best seller`는 무엇인가?

In [69]:
tr.상품중분류명.value_counts() #가장 많이팔린 순서대로 나오게됨 

가공식품              19385
여성용의류-SPA         12373
농산물                8492
스포츠용품/의류           6230
화장품                5320
주방가전               5112
음료                 4173
여성용의류-정장류          4170
남성용의류-캐주얼          3164
생활가전               3029
구두                 2993
수산물                2970
생활용품               2282
축산물                2219
패션잡화               2120
아동용의류              2098
남성용의류-정장류          2050
골프용품/의류            1785
여성용의류-고급의류         1414
여성용의류-이너웨어         1410
건강식품               1358
침구류                1308
명품                 1226
주방용품               1176
TV/AV              1153
컴퓨터주변기기             980
가구                  734
휴대폰/태블릿             419
PC/노트북/프린터/카메라      313
주얼리                 122
여성용의류-캐주얼           114
Name: 상품중분류명, dtype: int64

In [70]:
tr.상품중분류명.value_counts().index[0]

'가공식품'

**[연습문제 33]** `축산물`은 하루 중 언제 가장 많이 팔리는가?

In [87]:
tr.query('상품중분류명== "축산물"').구매시간.value_counts().index[0] 
#순서대로 뽑은(value_counts) 다음에 첫번째꺼 0번째 (Index[0])

18

**[연습문제 34]** 전 지역에서 `판매량이 가장 많은 상품`의 `총 매출액`은 얼마인가?

In [72]:
#가공식품의 판매갯수 모두  뽑아서 가격 총 계산하면 문제풀이 가능 
tr.query('상품중분류명== "가공식품"').구매금액.sum()  
#이미 가공식품이 판매량 짱이란걸 알기에 이렇게계산 가능 모를지에 는 다르게 계산해야함 

218379000

In [75]:
#x를 변수선언 
x=tr.상품중분류명.value_counts().index[0]

tr.query('상품중분류명== @x').구매금액.sum() #@변수를 하면 추출가능 

218379000

**[연습문제 35]** `60대 고객`의 `총 구매액`은 얼마인가?

In [ ]:
#cs와 tr 합한 후 query 문 작성해야함 

In [110]:
pd.merge(cs,tr,on='ID').query('60<=연령 <=69').구매금액.sum()

2862716000

In [ ]:
#고객별 총 환불액 구해보자. 

In [120]:
cs.shape,cancel.shape #cs의 갯수 3542, cs에서 취소여부의 갯수 1553

((3542, 4), (1553, 2))

In [117]:
cancel= tr.query('취소여부 ==1').groupby('ID')['구매금액'].sum().reset_index()
cancel

,ID,구매금액
0,909249,-1149000
1,2272647,-1153000
2,2413953,-61000
3,2960045,-18000
4,3699906,-797000
...,...,...
1548,997853190,-2030000
1549,997894186,-2438000
1550,999184958,-124000
1551,999375162,-10754000


In [125]:
pd.merge(cs,cancel,how='left').fillna(0) 

,ID,성별,연령,거주지역,구매금액
0,478207946,1,84,서울 성동구,0.0
1,479806984,1,84,서울 서초구,0.0
2,94790213,1,84,부산 사상구,0.0
3,656026338,1,84,서울 영등포구,0.0
4,433076833,1,83,경기 고양시,-18000.0
...,...,...,...,...,...
3537,154314532,1,66,부산 사하구,0.0
3538,416998709,1,59,경기 수원시,0.0
3539,168692065,2,63,경기 안양시,0.0
3540,955481370,2,58,경기 의왕시,0.0


**[연습문제 36]** 각 `고객` 별로 `월` 별 `구매량`를 계산하여 출력하시오.

In [58]:
#tr 표의 행에 월이라는 행추가하기 
tr['월']= tr.구매일자// 100  %100 

In [65]:
pd.pivot_table(tr,
              index = 'ID',
              columns =  '월',
              values = '구매수량',
              aggfunc= np.sum,
              fill_value=0).reset_index()

월,ID,1,2,3,4,5,6,7,8,9,10,11,12
0,741463,1,2,3,1,0,0,0,0,0,0,0,0
1,909249,0,1,0,4,3,2,0,6,0,0,0,2
2,1065068,0,0,0,0,0,0,2,0,0,5,0,4
3,1310410,2,0,0,0,0,0,0,0,0,1,0,0
4,1387032,1,2,0,0,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3537,998623524,0,4,1,0,4,3,0,0,3,0,1,5
3538,999024423,0,0,0,0,1,0,0,0,0,1,1,0
3539,999184958,3,0,2,0,7,3,0,7,0,2,0,0
3540,999375162,2,0,1,16,5,0,5,0,0,0,0,5


In [ ]:
#고객별 아니고 상품대분류명, 상품 소분류명 등으로 index 정렬 가능함 

<font color = "#CC3D3D"><p>
# End